<a href="https://colab.research.google.com/github/cloner174/mr.DGA/blob/main/mr_DGA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget 'https://www.filetolink.xyz/stream?id=W2FewpfCqMOlwqLCnsKbwqHCiVpewr7CqMKiwpDCoMKbwqdcWmTCj8Kuw6XDlsOgw47Dkg==&filename=dataset.csv'

--2024-04-30 00:14:26--  https://www.filetolink.xyz/stream?id=W2FewpfCqMOlwqLCnsKbwqHCiVpewr7CqMKiwpDCoMKbwqdcWmTCj8Kuw6XDlsOgw47Dkg==&filename=dataset.csv
Resolving www.filetolink.xyz (www.filetolink.xyz)... 104.21.94.124, 172.67.223.113, 2606:4700:3036::6815:5e7c, ...
Connecting to www.filetolink.xyz (www.filetolink.xyz)|104.21.94.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 300767719 (287M) [text/csv]
Saving to: ‘stream?id=W2FewpfCqMOlwqLCnsKbwqHCiVpewr7CqMKiwpDCoMKbwqdcWmTCj8Kuw6XDlsOgw47Dkg==&filename=dataset.csv’

stream?id=W2FewpfCq 100%[===================>] 286.83M  4.65MB/s    in 66s     

2024-04-30 00:15:34 (4.33 MB/s) - ‘stream?id=W2FewpfCqMOlwqLCnsKbwqHCiVpewr7CqMKiwpDCoMKbwqdcWmTCj8Kuw6XDlsOgw47Dkg==&filename=dataset.csv’ saved [300767719/300767719]



In [2]:
%pip install imutils opencv-python dlib opencv-python-headless pillow matplotlib face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=c4391ac0890c3b7c781f8e7cc4693b39cc672fb3972bbb0691064169afc78ea0
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [3]:
import pandas as pd
import cv2
import pandas as pd
import face_recognition
import numpy as np
import os

In [4]:
def strings_with_number_inside( data,
                                    column_range_starts : int = 0,
                                    column_range_ends : int = None,
                                    seperator = None):
        """
        Converts cells with type String contains a series of numbers separated with spaces to a Dictionary and Gives it back.
        """
        start = column_range_starts
        end = int( data.shape[1] ) if column_range_ends is None else column_range_ends  #Columns
        # Split string and convert to uint8 numpy array
        new_data = {}
        if isinstance( data , pd.DataFrame ) :
            for i in range( data.shape[0] ) :
                temp_new_data = []
                for j in range( start, end ) :
                    # for each cell
                    # Any exception should be modify by column_range_starts and column_range_ends!
                    cells_real_values = np.array(data.iloc[i, j].split(sep = seperator), dtype='uint8')
                    temp_new_data.append(cells_real_values)
                new_data[i] = temp_new_data

            return new_data
        else:
            if isinstance( data , np.array ) :
                pass
            else:
                raise TypeError( " data just could be  PandasDataFrame  or  NumpyArray  ! ")

In [6]:
# Load data
csv_path = '/content/stream?id=W2FewpfCqMOlwqLCnsKbwqHCiVpewr7CqMKiwpDCoMKbwqdcWmTCj8Kuw6XDlsOgw47Dkg==&filename=dataset.csv'
data = pd.read_csv(csv_path)

In [7]:
# Path to save images
face_directory = 'data/face_directory/'
non_face_directory = 'data/non_face/'

In [8]:
data_pix = strings_with_number_inside(data, column_range_starts = 1)

In [9]:


# Define the image size
image_size = (48, 48)  # height, width

# Ensure directories exist
os.makedirs(face_directory, exist_ok=True)
os.makedirs(non_face_directory, exist_ok=True)

# loop to process images
for index, row in data_pix.items():
    # Convert the row to a grayscale image
    image_array = np.array(row, dtype=np.uint8).reshape(image_size[0], image_size[1])

    # In face_recognition, the image should be in rgb even if it's grayscale
    image_rgb = np.stack((image_array,) * 3, axis=-1)

    # Detect faces in the image using face_recognition
    face_locations = face_recognition.face_locations(image_rgb, model='cnn')  # using CNN model

    # Process each face found
    for face_location in face_locations:
        top, right, bottom, left = face_location
        face_image = image_array[top:bottom, left:right]  # Cropping the grayscale image directly
        cv2.imwrite(os.path.join(face_directory, f'face_{index}.jpg'), face_image)

    # If no faces are detected, save the full image to the non-face directory
    if len(face_locations) == 0:
        cv2.imwrite(os.path.join(non_face_directory, f'non_face_{index}.jpg'), image_array)

print("Face cropping and saving completed.")


Face cropping and saving completed.


In [10]:
in_face_folder = os.listdir('/content/data/face_directory')

In [12]:
non_face_folder = os.listdir('/content/data/non_face')

In [11]:
len(in_face_folder)

31519

In [13]:
len(non_face_folder)

4368

In [ ]:
!zip -r '/content/face_directory.zip' '/content/data/face_directory/'

In [ ]:
!zip -r '/content/non_face.zip' '/content/data/non_face'